Lauri Maila <br>
Student number 2209361 <br>
February, 6, 2024  <br>

# Exercise 2 | TKO_7092 Evaluation of Machine Learning Methods 2024

### Prediction of the metal ion content from multi-parameter data
<b> Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), using number of neighbors k = 1, 3, 5, 7.</b> <br>

<b> Instructions: </b> 

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score (using sklearn.preprocessing.StandardScaler is advised).
    - Implement Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Explain your code by adding detailed comments. 
    - Only provide code that is relevant to the exercise.
    - Please submit your solution as a Jupyter Notebook (.ipynb) and as a PDF file. Ensure to include your full name in the filename.     
    - Submit to moodle your solution on ** Wednesday 7 of February ** at the latest.
  
<b>Please follow the instructions and note that you are expected to submit your individual solution.<br>
Identical or overly similar submissions will result in the exercise being marked as failed.
</b>

## Import libraries

In [12]:
# In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

## Read and visualize the dataset

In [13]:
# In this cell read the file Water_data.csv
waterdata_df = pd.read_csv('water_data.csv')
dim = waterdata_df.shape
print("Number of rows:", dim[0])
print("Number of columns:", dim[1])
waterdata_df.head()

Number of rows: 268
Number of columns: 6


,Mod1,Mod2,Mod3,c_total,Cd,Pb
0,9945,119,72335,0,0.0,0.0
1,9596,119,110542,0,0.0,0.0
2,10812,120,98594,0,0.0,0.0
3,10786,117,82977,0,0.0,0.0
4,10566,108,136416,14,0.0,14.0


## Standardization of the dataset

In [14]:
# In this cell, standardize the dataset features by removing the mean and scaling to unit variance. 
# In other words, use z-score to scale the dataset features (Mod1, Mod2, Mod3) 
# Print the 5 first samples (i.e. rows) of the scaled dataset

# We'll use sklearn StandardScaler
scaler = StandardScaler()

# Fit and transform the chosen features
scaled_features = scaler.fit_transform(waterdata_df[['Mod1', 'Mod2', 'Mod3']])

# Create a DataFrame with the scaled features
scaled_waterdata_df = pd.DataFrame(scaled_features, columns=['Mod1', 'Mod2', 'Mod3'])

# Print the first 5 samples of the scaled dataset
scaled_waterdata_df.head()

,Mod1,Mod2,Mod3
0,-0.972283,-0.670482,-0.358179
1,-0.975878,-0.670482,0.259488
2,-0.963351,-0.670394,0.066333
3,-0.963619,-0.670657,-0.186137
4,-0.965885,-0.671447,0.677776


## C-index code 

In [15]:
# I'm using the C-index implementation given in the first exercise
"""
C-index function: 
- INPUTS: 
'y' an array of the true output values
'yp' an array of predicted output values
- OUTPUT: 
The c-index value
"""
def cindex(y, yp):
    n = 0
    h_num = 0 
    for i in range(0, len(y)):
        t = y[i]
        p = yp[i]
        for j in range(i+1, len(y)):
            nt = y[j]
            np = yp[j]
            if (t != nt): 
                n = n + 1
                if (p < np and t < nt) or (p > np and t > nt): 
                    h_num += 1
                elif (p == np):
                    h_num += 0.5
    return h_num/n

In [16]:
# Test the cindex function with following values
true_labels = np.array([-1, 1, 1, -1, 1])
predictions = np.array([0.60, 0.80, 0.75, 0.75, 0.70])
cindx = cindex(true_labels, predictions)
print(cindx) #For this example, a correct C-index implementation will result in 0.75

0.75


## Leave-One-Out cross-validation
In the following cell, write and execute your code for Leave-One-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-One-Out C-index for c_total, Cd and Pb for each k value.

In [21]:
X = scaled_waterdata_df[['Mod1', 'Mod2', 'Mod3']]
y = waterdata_df[['c_total', 'Cd', 'Pb']]

# Values of k to try
k_values = [1, 3, 5, 7]

# Iterate over each target variable
for target_col in y.columns:
    print(f"Target variable: {target_col}")
    # Iterate over each k value
    for k in k_values:
        loocv = LeaveOneOut()
        total_cindex = 0
        
        # Perform LOOCV
        for train_index, test_index in loocv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[target_col].iloc[train_index], y[target_col].iloc[test_index]
            
            # Initialize KNN regressor
            knn = KNeighborsRegressor(n_neighbors=k)
            knn.fit(X_train, y_train)
            
            # Predict on the test data
            y_pred = knn.predict(X_test)
            
            # Calculate C-index
            total_cindex += cindex(y_test.values, y_pred)
        
        # Calculate average C-index
        avg_cindex = total_cindex / len(X)
        print(f"K = {k}: Leave-One-Out C-index: {avg_cindex:.4f}")

Target variable: c_total


ZeroDivisionError: division by zero

## Leave-Replicas-Out cross-validation

In the following cell, write and execute your code for Leave-Replicas-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-Replicas-Out C-index for c_total, Cd and Pb for each k value.

## Plot Leave-One-Out and Leave-Replicas-Out Results

Note: You may plot the results as they were presented in the video lecture (refer to MOOC2-Module 2 .pptx slides).

## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation method had more optimistic results? 
- Explain the reason for the optimistic results produced by the cross-validation method.
- Which cross-validation method generalized better on unseen data? Why?

In [10]:
#In this cell write your answers to the questions.